In [22]:
# импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [23]:
# Загрузка данных с ценами на ноутбуки и их характеристиками
data = pd.read_csv('./Laptop-Price.csv')
data.head()

,Company,Product,TypeName,Inches,ScreenResolution,Ram,OpSys,Cpu Brand,Cpu Model,Cpu Rate,SSD,HDD,Flash Storage,Hybrid,Gpu Brand,Gpu Model,Unknown,Price_euros
0,HP,250 G6,Notebook,15.6,Full HD 1920x1080,8GB,No OS,Intel,Core i5 7200U,2.5GHz,256,0,0,0,Intel,HD Graphics 620,NaN,575.0
1,Acer,Aspire 3,Notebook,15.6,1366x768,4GB,Windows 10,AMD,A9-Series 9420,3GHz,0,500,0,0,AMD,Radeon R5,NaN,400.0
2,Asus,ZenBook UX430UN,Ultrabook,14.0,Full HD 1920x1080,16GB,Windows 10,Intel,Core i7 8550U,1.8GHz,512,0,0,0,Nvidia,GeForce MX150,NaN,1495.0
3,Acer,Swift 3,Ultrabook,14.0,IPS Panel Full HD 1920x1080,8GB,Windows 10,Intel,Core i5 8250U,1.6GHz,256,0,0,0,Intel,UHD Graphics 620,NaN,770.0
4,HP,250 G6,Notebook,15.6,1366x768,4GB,No OS,Intel,Core i5 7200U,2.5GHz,0,500,0,0,Intel,HD Graphics 620,NaN,393.9


In [24]:
# Бесполезный признак
data = data.drop('Unknown', axis=1)
data.isnull().sum()

# Матрица признаков
X = data.drop('Price_euros', axis=1)
# Зависимая переменная
y = data['Price_euros']

In [25]:
# Обработка категориальных данных
categorical_features = [
  'Company',
  'Product',
  'TypeName',
  'OpSys',
  'Cpu Brand',
  'Gpu Brand',
  'ScreenResolution',
  'Cpu Model',
  'Gpu Model',
]

from sklearn.preprocessing import LabelEncoder

X['Ram'] = X['Ram'].astype(str).str.replace('GB', '', regex=False)
X['Cpu Rate'] = X['Cpu Rate'].astype(str).str.replace('GHz', '', regex=False)

for feature in categorical_features:
  labelencoder = LabelEncoder()
  X[feature] = labelencoder.fit_transform(X[feature])

X.head(10)

,Company,Product,TypeName,Inches,ScreenResolution,Ram,OpSys,Cpu Brand,Cpu Model,Cpu Rate,SSD,HDD,Flash Storage,Hybrid,Gpu Brand,Gpu Model
0,6,46,3,15.6,7,8,3,1,30,2.5,256,0,0,0,1,49
1,0,52,3,15.6,0,4,4,0,11,3,0,500,0,0,0,74
2,2,517,4,14.0,7,16,4,1,53,1.8,512,0,0,0,2,38
3,0,370,4,14.0,14,8,4,1,37,1.6,256,0,0,0,1,95
4,6,46,3,15.6,0,4,3,1,30,2.5,0,500,0,0,1,49
5,6,46,3,15.6,7,4,3,1,21,2,0,500,0,0,1,44
6,3,204,3,15.6,7,4,4,1,21,2,256,0,0,0,0,81
7,3,204,3,15.6,7,8,4,1,45,2.7,256,0,0,0,0,81
8,9,177,3,15.6,7,8,3,1,23,2.4,0,1,0,0,2,16
9,3,488,4,13.3,12,8,4,1,37,1.6,128,0,0,0,1,95


In [26]:
from sklearn.model_selection import train_test_split

# Разделение выборки на тестовую и тренировочную
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [27]:
from sklearn.linear_model import LinearRegression

# Обучшение линейной модели
regression = LinearRegression()
regression.fit(X_train, y_train)

LinearRegression()

In [28]:
# предсказывание тестовых данных
y_pred = regression.predict(X_test)
regression.score(X_test, y_test)

0.7216324598543796

In [30]:
# Проверка пустых значений
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

In [32]:
# Оптимизация модели
import statsmodels.api as sm

X_train_imputed = sm.add_constant(X_train_imputed)
model = sm.OLS(y_train, X_train_imputed).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Price_euros   R-squared:                       0.646
Model:                            OLS   Adj. R-squared:                  0.639
Method:                 Least Squares   F-statistic:                     102.4
Date:                Sun, 29 Sep 2024   Prob (F-statistic):          9.07e-190
Time:                        11:44:00   Log-Likelihood:                -6779.9
No. Observations:                 916   AIC:                         1.359e+04
Df Residuals:                     899   BIC:                         1.368e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1078.7459    268.564     -4.017      0.000   -1605.831    -551.660
x1             8.7459      3.691      2.370      0.018       1.502      15.990
x2             0.1336      0.096      1.396      0.163      -0.054       0.322
x3            27.0941     11.292      2.399      0.017       4.932      49.256
x4           -37.3346     12.495     -2.988      0.003     -61.858     -12.811
x5             7.7380      2.415      3.204      0.001       2.999      12.477
x6            70.7264      3.348     21.125      0.000      64.155      77.297
x7           113.5676     17.326      6.555      0.000      79.563     147.572
x8           243.2953     76.226      3.192      0.001      93.693     392.898
x9             6.3696      1.446      4.404      0.000       3.531       9.208
x10          260.7810     37.726      6.912      0.000     186.740     334.822
x11            0.6284      0.106      5.931      0.000       0.420       0.836
x12            0.2325      0.105      2.207      0.028       0.026       0.439
x13            1.7473      0.519      3.369      0.001       0.730       2.765
x14            0.1952      0.795      0.245      0.806      -1.365       1.756
x15          151.1961     34.524      4.379      0.000      83.438     218.954
x16            3.4425      0.955      3.606      0.000       1.569       5.316
==============================================================================
Omnibus:                      365.487   Durbin-Watson:                   2.031
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2622.909
Skew:                           1.647   Prob(JB):                         0.00
Kurtosis:                      10.607   Cond. No.                     7.66e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.66e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

> [!NOTE]
> Переменные c p-значениями более 0.05, то такая переменная может быть считаться не значительной и убрана из модели, таким образом можно убрать переменные x2 и x14

In [33]:
# Backward elimination не значительных признаков
data = data.drop('Product', axis=1)
data = data.drop('Hybrid', axis=1)